In [1]:
import sys
sys.path.append('../dl1')
import os
import shutil
import h5py
from eventlist_v4 import Eventlist
from snapeventlist import EventlistSnapshot
import matplotlib.pyplot as plt
import tables as tb
from tqdm import tqdm
plt.rcParams['figure.figsize'] = [25, 10]

## __Test__
* DLO: `wf_runId_00115_configId_00000_2022-10-03T22_39_08.295757.h5`
    * single event: `dl0_id=18` and `dl1_id=18`
    * double event: `dl0_id=315` and `dl1_id=315`


In [2]:
# Define DL folder path
dl0_path      = os.path.join('..', 'dl1')
dl1_path      = os.path.join('..', '..', 'Data', 'DL1')
dl1todl2_path = os.path.join('..', '..', 'Data', 'DL1toDL2')
dl0todl2_path = os.path.join('..', '..', 'Data', 'DL0toDL2')

In [3]:
# Define file path
dl0_file     = os.path.join(dl0_path, 'prova.h5')
dl0tol2_file = os.path.join(dl0todl2_path, 'prova.dl2.h5')
dl1_file     = os.path.join(dl1_path, 'prova.dl1.h5')
dl1tol2_file = os.path.join(dl1todl2_path, 'prova.dl1.dl2.h5')

In [4]:
# Config and XML model
config_file  = os.path.join('..', 'dl1', 'dl02dl1_config.json')
dl1model_file = os.path.join('..', 'dl1', 'DL1model.xml')

In [5]:
snapeventlist = EventlistSnapshot(config_file, dl1model_file)
eventlist_dl0 = Eventlist()
eventlist_dl1 = Eventlist(from_dl1=True, 
                          xml_model_path=dl1model_file)

In [6]:
# start = 1839
# end   = 1839
start = 0
end  = -1
if start == 0 and end == -1:
    log = False
else:
    log = True

### __DL0 to DL2__

In [7]:
if os.path.exists(dl0tol2_file):
    os.remove(dl0tol2_file)
eventlist_dl0.process_file(dl0_file, None, dl0todl2_path, log, start, end)

Processing ../dl1/prova.h5
Il file '../../Data/DL0toDL2/prova.dl2.h5' non esiste.


  2%|▏         | 197/10000 [00:00<00:30, 322.58it/s]

100%|██████████| 10000/10000 [00:31<00:00, 321.67it/s]


### __DL0 to DL1__

In [8]:
if os.path.exists(dl1_file):
    os.remove(dl1_file)
snapeventlist.process_file(dl0_file, dl1_path, start, end)

  0%|          | 31/10000 [00:00<00:32, 304.78it/s]

100%|██████████| 10000/10000 [00:30<00:00, 326.94it/s]


wfs
attrs_gps
attrs_reco_int
attrs_reco_float
attrs_int


### __DL1 to DL2__

In [9]:
if os.path.exists(dl1tol2_file):
    os.remove(dl1tol2_file)
eventlist_dl1.process_file(dl1_file, None, dl1todl2_path, log, start, end)

Processing ../../Data/DL1/prova.dl1.h5
Il file '../../Data/DL1toDL2/prova.dl1.dl2.h5' non esiste.


  0%|          | 45/10126 [00:00<00:22, 442.28it/s]

100%|██████████| 10126/10126 [00:21<00:00, 464.30it/s]


### __Check if the two DL2 file are the same__

In [10]:
# filePath_dl0_old_dl2 = os.path.join(outputdir0_old, os.path.basename(filePath_dl0)).replace('h5', 'dl2.h5')
# filePath_dl0_dl2     = os.path.join(outputdir0, os.path.basename(filePath_dl0)).replace('h5', 'dl2.h5')
# filePath_dl1_dl2     = os.path.join(outputdir1, os.path.basename(filePath_dl1)).replace('h5', 'dl2.h5')
# print(filePath_dl0_old_dl2)
# print(filePath_dl0_dl2)
# print(filePath_dl1_dl2)

### __DL2 of DL0 vs DL2 of DL1__

In [11]:
cnt_pk_notfound_h50 = 0
cnt_pk_notfound_h51 = 0
cnt_len_h50 = 0
cnt_len_h51 = 0
with h5py.File(dl0tol2_file, mode='r') as h50:
    cnt_len_h50 = len(h50['dl2']['eventlist'])
    evlst0 = h50['dl2']['eventlist']
    for data0 in evlst0:
        if -1 in data0:
            cnt_pk_notfound_h50 += 1
with h5py.File(dl1tol2_file, mode='r') as h51:
    cnt_len_h51 = len(h51['dl2']['eventlist'])
    evlst1 = h51['dl2']['eventlist']
    for data1 in evlst1:
        if -1 in data1:
            cnt_pk_notfound_h51 += 1

print(f"len h5_0=            {cnt_len_h50}")
print(f"len h5_1=            {cnt_len_h51}")
print(f"pks_not_found h5_0=  {cnt_pk_notfound_h50}")
print(f"pks_not_found h5_1=  {cnt_pk_notfound_h51}")

len h5_0=            10133
len h5_1=            10133
pks_not_found h5_0=  63
pks_not_found h5_1=  63


In [12]:
with h5py.File(dl0tol2_file, mode='r') as h5_0:
    with h5py.File(dl1tol2_file, mode='r') as h5_1:
        i, j = 0, 0
        evlst0 = h5_0['dl2']['eventlist']
        evlst1 = h5_1['dl2']['eventlist']
        for data0, data1 in tqdm(zip(evlst0, evlst1), total=cnt_len_h50):
            # print('dl0_2_dl2 tstart:', data0['tstart'])
            # print('dl1_2_dl2 tstart:', data1['tstart'])
            # print('-------------')
            if not data0==data1:
                print('dl0_2_dl2:', data0)
                print('dl1_2_dl2:', data1)
                print('dl0_2_dl2 tstart:', data0['tstart'])
                print('dl1_2_dl2 tstart:', data1['tstart'])
                print(data0['n_waveform'], data1['n_waveform'])
                raise Exception('stop')

100%|██████████| 10133/10133 [00:01<00:00, 5287.59it/s]
